In [1]:
import torch as th


In [2]:
x = th.tensor([1,2,3,4,5])

In [3]:
x


tensor([1, 2, 3, 4, 5])

In [4]:
import syft as sy

/home/akshay/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
hook = sy.TorchHook(th)

In [6]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

In [7]:
bob = sy.VirtualWorker(hook,id="Bob")

In [8]:
x=x.send(bob)


In [9]:
bob._objects

{63583798035: tensor([1, 2, 3, 4, 5])}

In [10]:
x

(Wrapper)>[PointerTensor | me:73784735819 -> Bob:63583798035]

In [11]:
x.location

<VirtualWorker id:Bob #objects:1>

In [12]:
x.id_at_location

63583798035

In [13]:
x.id

73784735819

In [14]:
x.owner

<VirtualWorker id:me #objects:0>

In [15]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [16]:
x

(Wrapper)>[PointerTensor | me:73784735819 -> Bob:63583798035]

In [17]:
x=x.get()
x

tensor([1, 2, 3, 4, 5])

In [18]:
bob._objects

{}

In [19]:
alice = sy.VirtualWorker(hook,id="alice")

In [20]:
x=th.tensor([1,2,3,4,5])

In [21]:
x_ptr = x.send(bob,alice)

In [22]:
x_ptr.get()

[tensor([1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5])]

In [23]:
x=th.tensor([1,2,3,4,5]).send(bob,alice)

In [24]:
x.get(sum_results=True)

tensor([ 2,  4,  6,  8, 10])

In [25]:
input = th.tensor([[1.,1],[0,1],[1,0],[0,0]],requires_grad= True).send(bob)
target = th.tensor([[1.],[1],[0],[0]], requires_grad=True).send(bob)

In [26]:
weights = th.tensor([[0.],[0.]],requires_grad= True).send(bob)

In [27]:
for i in range(10):
    pred = input.mm(weights)
    loss = ((pred-target)**2).sum()
    loss.backward()
    weights.data.sub_(weights.grad*0.1)
    weights.grad*=0
    print(loss.get().data)

tensor(2.)
tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)


In [28]:
bob = bob.clear_objects()

In [29]:
bob._objects


{}

In [30]:
x=th.tensor([1,2,3,4,5]).send(bob)

In [31]:
bob._objects

{40507855632: tensor([1, 2, 3, 4, 5])}

In [32]:
del x


In [33]:
bob._objects

{}

In [34]:
from torch import nn,optim

In [35]:
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]],requires_grad= True)
target = th.tensor([[1.],[1],[0],[0]], requires_grad=True)

In [36]:
model = nn.Linear(2,1)

In [37]:
opt = optim.SGD(params=model.parameters(),lr=0.1)

In [38]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()
        pred=model(data)
        loss=((pred-target)**2).sum()
        loss.backward()
        opt.step()
        print(loss.data)
train()

tensor(4.7918)
tensor(1.6687)
tensor(0.9772)
tensor(0.6297)
tensor(0.4119)
tensor(0.2708)
tensor(0.1788)
tensor(0.1187)
tensor(0.0792)
tensor(0.0531)
tensor(0.0359)
tensor(0.0244)
tensor(0.0167)
tensor(0.0116)
tensor(0.0080)
tensor(0.0056)
tensor(0.0040)
tensor(0.0028)
tensor(0.0020)
tensor(0.0015)


In [39]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [40]:
data_alice = data[0:2].send(alice)
target_alice= target[0:2].send(alice)

In [41]:
datasets=[(data_bob,target_bob),(data_alice,target_alice)]

In [47]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            
            model = model.send(_data.location)
            
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()
            model = model.get()

            print(loss.get())

In [48]:
train()

tensor(11.5229, requires_grad=True)
tensor(0.0960, requires_grad=True)
tensor(0.0069, requires_grad=True)
tensor(0.0052, requires_grad=True)
tensor(0.0043, requires_grad=True)
tensor(0.0036, requires_grad=True)
tensor(0.0030, requires_grad=True)
tensor(0.0025, requires_grad=True)
tensor(0.0021, requires_grad=True)
tensor(0.0017, requires_grad=True)
tensor(0.0014, requires_grad=True)
tensor(0.0012, requires_grad=True)
tensor(0.0010, requires_grad=True)
tensor(0.0008, requires_grad=True)
tensor(0.0007, requires_grad=True)
tensor(0.0006, requires_grad=True)
tensor(0.0005, requires_grad=True)
tensor(0.0004, requires_grad=True)
tensor(0.0003, requires_grad=True)
tensor(0.0003, requires_grad=True)
tensor(0.0002, requires_grad=True)
tensor(0.0002, requires_grad=True)
tensor(0.0002, requires_grad=True)
tensor(0.0001, requires_grad=True)
tensor(0.0001, requires_grad=True)
tensor(9.1586e-05, requires_grad=True)
tensor(7.6228e-05, requires_grad=True)
tensor(6.3445e-05, requires_grad=True)
tensor(